In [37]:
import pandas as pd
from tqdm import tqdm_notebook

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [2]:
#
df_tripchain = pd.read_csv('./data/PJT001_TripChain.csv')
df_tripchain.shape

(1048575, 51)

In [3]:
#
df_tripchain.head(10)


,암호화카드번호||','||트랜잭션ID||','||환승횟수||','||교통카드발행사ID||','||총이용객수||','||사용자구분||','||교통수단CD1||','||교통수단CD2||','||교통수단CD3||','||교통수단CD4||','||교통수단CD5||','||버스노선ID1||','||버스노선ID2||','||버스노선ID3||','||버스노선ID4||','||버스노선ID5||','||차량ID1||','||차량ID2||','||차량ID3||','||차량ID4||','||차량ID5||','||총통행거리||','||총탑승시간||','||총소요시간||','||승차일시1||','||승차일시2||','||승차일시3||','||승차일시4||','||승차일시5||','||하차일시1||','||하차일시2||','||하차일시3||','||하차일시4||','||하차일시5||','||최초승차일시||','||최종하차일시||','||승차역ID1||','||승차역ID2||','||승차역ID3||','||승차역ID4||','||승차역ID5||','||하차역ID1||','||하차역ID2||','||하차역ID3||','||하차역ID4||','||하차역ID5||','||최초승차역ID||','||최종하차역ID||','||총이용금액||','||수집건수||','||트립체인완료코드
0,900079696430,56,2,9000923,1,1,500.0,500.0,NaN,NaN,NaN,41002045.0,41002044.0,NaN,NaN,NaN,141771735.0,141771587.0,NaN,NaN,NaN,11170,25,25,2.018070e+13,2.018070e+13,NaN,NaN,NaN,2.018070e+13,2.018070e+13,NaN,NaN,NaN,20180701052543,20180701064826,4116828.0,4150144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4116708.0,1350,2,;
1,900079697651,5,1,9000923,1,1,500.0,NaN,NaN,NaN,NaN,41031040.0,NaN,NaN,NaN,NaN,141701792.0,NaN,NaN,NaN,NaN,1700,3,3,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701072156,20180701072520,4117280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4117269.0,1250,1,;
2,900079698254,32,1,9000923,1,1,500.0,NaN,NaN,NaN,NaN,41031121.0,NaN,NaN,NaN,NaN,141701843.0,NaN,NaN,NaN,NaN,23180,66,66,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701123653,20180701134223,4199619.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4107936.0,1550,1,;
3,900079699257,80,1,9000923,1,1,500.0,NaN,NaN,NaN,NaN,41031013.0,NaN,NaN,NaN,NaN,141701450.0,NaN,NaN,NaN,NaN,500,1,1,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701224424,20180701224543,4108130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4116717.0,1250,1,;
4,900079701419,64,1,9000923,1,1,530.0,NaN,NaN,NaN,NaN,41020001.0,NaN,NaN,NaN,NaN,141703985.0,NaN,NaN,NaN,NaN,3240,8,8,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701085058,20180701085910,4100122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4116848.0,2050,1,;
5,900079701419,65,1,9000923,1,1,530.0,NaN,NaN,NaN,NaN,41020001.0,NaN,NaN,NaN,NaN,141703953.0,NaN,NaN,NaN,NaN,2940,8,8,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701220922,20180701221723,4100098.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4100121.0,2050,1,;
6,900079705567,86,3,9000923,1,1,511.0,202.0,500.0,NaN,NaN,41067109.0,NaN,41021002.0,NaN,NaN,141763724.0,999999999.0,141711728.0,NaN,NaN,24480,39,39,2.018070e+13,2.018070e+13,2.018070e+13,NaN,NaN,2.018070e+13,2.018070e+13,2.018070e+13,NaN,NaN,20180701075731,20180701085359,4197606.0,1716.0,4116611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4116627.0,1550,3,;
7,900079705567,87,3,9000923,1,1,120.0,202.0,511.0,NaN,NaN,41110050.0,NaN,41067109.0,NaN,NaN,999999999.0,999999999.0,141763724.0,NaN,NaN,24490,41,41,2.018070e+13,2.018070e+13,2.018070e+13,NaN,NaN,2.018070e+13,2.018070e+13,2.018070e+13,NaN,NaN,20180701154641,20180701163919,9814.0,1708.0,4151651.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4197607.0,1550,3,;
8,900079969054,43,1,9000923,1,1,500.0,NaN,NaN,NaN,NaN,41002045.0,NaN,NaN,NaN,NaN,141771381.0,NaN,NaN,NaN,NaN,3670,11,11,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701123115,20180701124235,4100527.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4100445.0,1250,1,;
9,900079971165,94,1,9000923,1,1,500.0,NaN,NaN,NaN,NaN,41036102.0,NaN,NaN,NaN,NaN,141705399.0,NaN,NaN,NaN,NaN,5340,22,22,2.018070e+13,NaN,NaN,NaN,NaN,2.018070e+13,NaN,NaN,NaN,NaN,20180701121554,20180701123726,4111874.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4108071.0,1250,1,;


### column명 재설정 필요

In [4]:
#
# mydict[new_key] = mydict.pop(old_key) 활용

keys = df_tripchain.keys()
keys[1][1:-1]
for i in range(1, len(keys)-1):
    df_tripchain[keys[i][3:-3]] = df_tripchain.pop('\''+keys[i][1:-1]+'\'')

In [5]:
#
df_tripchain['암호화카드번호'] = df_tripchain.pop('암호화카드번호||\'')
df_tripchain['트립체인완료코드'] = df_tripchain.pop('\'||트립체인완료코드')

In [114]:
#
df_tripchain.head(5)

601471

날짜 포멧이 제 각각이다.  
모두 datetime 포맷으로 바꾸면, 용량이 너무 커질 것이므로 필요한 열인 최초 하차일시만 datetime 형태로 바꿈.

In [7]:
#
df_tripchain['최종하차일시'] = pd.to_datetime(df_tripchain['최종하차일시'], errors = 'coerce')

버스 이용시간을 버스 최종하차 기준으로 잡으려고 한다.  
따라서 최종하차일시가 결측된 데이터는 데이터에서 제거해준다.

In [8]:
#
before = len(df_tripchain)
df_tripchain.dropna(subset = ['최종하차일시'], inplace = True)
after = len(df_tripchain)

print("{} -> {} ({:.2f})".format(before, after, (before - after)/before))

1048575 -> 1036746 (0.01)


1% 데이터만 소실됐다.

## 특정 날짜 데이터만 가져오기

이제 몇 일의 날짜들만 있는지 확인


In [9]:
df_tripchain['최종하차일시'].dt.day.value_counts().sort_index()

1    177387
2    360373
3    399390
4     98971
5       625
Name: 최종하차일시, dtype: int64

  
2018년 7월 1일 ~ 5일 까지의 데이터가 있음을 확인 (하차날짜 기준)  
이 중에서, 데이터가 가장 많은 **2018년 7월 3일** 데이터만 사용  

In [10]:
df = df_tripchain[df_tripchain['최종하차일시'].dt.day == 3].reset_index(drop = True)

이 데이터의 핵심은 승차와 하차에 대한 정보가 담겨있는 것이다.  
위 데이터를 잘 살펴보면 최초 승차역 ID가 대부분 결측값이지만, 승차역 ID1가 대체로 동일함을 알 수 있다.  
따라서, 최초승차역 ID의 값들을 다음과 같이 채워준다.

In [11]:
df['최초승차역ID'] = df['승차역ID1']

승차역과 하차역에 대한 결측치를 살펴본다.

In [12]:
print(df['최초승차역ID'].isna().sum())
print(df['최종하차역ID'].isna().sum())

16
1135


전체에 비하면 매우 적은양이기 때문에 결측 값을 가지는 데이터는 모두 드랍.

In [22]:
before = len(df)
df.dropna(subset=['최초승차역ID', '최종하차역ID'], inplace=True)
after = len(df)

print("{} -> {} ({:.2f})".format(before, after, (before-after)/before))

398249 -> 398249 (0.00)


드랍된 데이터가 1% 미만임을 알 수 있다.

## OD 시각화 by 정류장
### 정류장 테이블과 합치기
승차 정류장에서 하차 정류장으로 가는 이동을 시각화 해보려고 한다.  

먼저 데이터 프레임을 만들어 준다

In [47]:
### 수집건수 분류 x ###
df_from_to = df.groupby(['최초승차역ID', '최종하차역ID']).size().to_frame().reset_index()
df_from_to.rename({0: "이용량"}, axis=1, inplace=True)
df_from_to.head(5)

,최초승차역ID,최종하차역ID,이용량
0,150.0,4100042.0,1
1,150.0,4100051.0,1
2,150.0,4100068.0,1
3,150.0,4100077.0,1
4,150.0,4100083.0,1


각 정류장 ID 에 대한 정보가 필요하다.  
따라서 정류장 테이블 데이터를 불러온다. 

In [48]:
df_stations = pd.read_csv('./data/PJT001_stations_table.csv')

In [49]:
df_stations.head(5)

,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID
0,228003422,용인시,손골마을회관.국제학교,"Songol Community Center,",56443.0,노변정류장,경기도 용인시,NaN,37.342517,127.066817,56443.0,NaN
1,228003423,용인시,풀잎사랑,Pulipsarang,56444.0,노변정류장,경기도 용인시,NaN,37.341800,127.068983,56444.0,NaN
2,228003424,용인시,풀잎사랑,Pulipsarang,56445.0,노변정류장,경기도 용인시,NaN,37.341817,127.069083,56445.0,NaN
3,228003425,용인시,대성공정,Daesung Process,56446.0,노변정류장,경기도 용인시,NaN,37.339350,127.073067,56446.0,NaN
4,228003426,용인시,대성공정,Daesung Process,56447.0,노변정류장,경기도 용인시,NaN,37.339183,127.073400,56447.0,NaN


In [50]:
### 수집건수 분류 x merge ###
df_from_to = pd.merge(df_from_to,
            df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
            left_on='최초승차역ID', right_on='이비카드정류장ID', how='left')

df_from_to = pd.merge(df_from_to, 
            df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
            left_on='최종하차역ID', right_on='이비카드정류장ID', how='left',
            suffixes=('_승차', '_하차'))
df_from_to.head(5)

,최초승차역ID,최종하차역ID,이용량,이비카드정류장ID_승차,정류소명_승차,표준정류장ID_승차,WGS84위도_승차,WGS84경도_승차,관할관청_승차,이비카드정류장ID_하차,정류소명_하차,표준정류장ID_하차,WGS84위도_하차,WGS84경도_하차,관할관청_하차
0,150.0,4100042.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150.0,4100051.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100051.0,동부출장소.병점초등학교,233000708.0,37.208800,127.034433,경기도 화성시
2,150.0,4100068.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100068.0,호계2동주민센터,209000126.0,37.382017,126.952783,경기도 안양시
3,150.0,4100077.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100077.0,비상활주로.곡반정동입구,201000040.0,37.233000,127.021367,경기도 수원시
4,150.0,4100083.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100083.0,수원버스터미널,201000009.0,37.250950,127.019550,경기도 수원시


이 중에는 위치정보에 결측치가 있을 수도 있다.

In [51]:
### 수집건수 분류 x 결측치 확인 ###
cols = [col for col in df_from_to.columns if "WGS" in col]
missing_num = df_from_to[cols].isna().any(axis=1).sum()

print("{} / {} ({:.2f}%)".format(missing_num, len(df_from_to), missing_num/len(df_from_to)))

72480 / 186048 (0.39%)


약 39%의 결측치가 있다.  
지도에 시각화 시킬 수 없으므로, 드랍시키도록 한다.

In [56]:
### 수집건수 분류 x 결측치 제거 ###
df_from_to = df_from_to[~df_from_to[cols].isna().any(axis=1)]
df_from_to.tail(5)
#관할관청 하차 경기도 화성시만 
#관할관청 승차 경기도 화성시만 

,최초승차역ID,최종하차역ID,이용량,이비카드정류장ID_승차,정류소명_승차,표준정류장ID_승차,WGS84위도_승차,WGS84경도_승차,관할관청_승차,이비카드정류장ID_하차,정류소명_하차,표준정류장ID_하차,WGS84위도_하차,WGS84경도_하차,관할관청_하차
185970,9196637.0,4100565.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4100565.0,파크타운.양지마을,206000183.0,37.373967,127.119567,경기도 성남시
185971,9196637.0,4100566.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4100566.0,중앙공원.샛별마을우방아파트,206000184.0,37.372633,127.126183,경기도 성남시
185972,9196637.0,4116808.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4116808.0,여권민원실.풍림아파트,202000054.0,37.287450,127.033083,경기도 수원시
185973,9196637.0,4150166.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4150166.0,벽산아파트,228001144.0,37.334750,127.108717,경기도 용인시
185974,9196865.0,4108038.0,1,9196865.0,인정피렌체아파트,228001295.0,37.246183,127.229500,경기도 용인시,4108038.0,망포블루밍.삼정라츠아파트,203000147.0,37.237183,127.059267,경기도 수원시


In [ ]:
### 수집건수 분류 x 데이터생성 ###
# df_from_to.to_csv('./data/ODdata3.csv')

In [122]:
routemapping = pd.read_csv('./data/PJT001_routestationmapping.csv')

In [125]:
routemapping

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
0,경기시내,경원여객M,2805000,216000044,28050900,M6410
1,경기시내,강화운수,4100100,232000028,41001001,2
2,경기시내,강화운수,4100100,232000029,41001013,88
3,경기시내,강화운수,4100100,232000061,41001020,3000
4,경기시내,강화운수,4100100,232000067,41001024,388
...,...,...,...,...,...,...
2122,경기시내,서현운수,4108800,229000060,41088004,330
2123,경기시내,서현운수,4108800,229000063,41088005,850
2124,경기시내,코레일네트웍스,4108900,213000024,41089001,8507
2125,경기시내,신성교통,4109100,229000102,41091900,M7111


In [150]:
# 수집건수 == 1 환승 없는 자료

# 수집건수 1
df_from_to1 = df[df['수집건수'] == 1].groupby(['최초승차역ID', '최종하차역ID', '버스노선ID1']).size().to_frame().reset_index()
df_from_to1.rename({0: "이용량"}, axis=1, inplace=True)
df_from_to1

,최초승차역ID,최종하차역ID,버스노선ID1,이용량
0,4100002.0,4100004.0,41027009.0,1
1,4100002.0,4100020.0,41027009.0,1
2,4100002.0,4100022.0,41020001.0,1
3,4100002.0,4100024.0,41027009.0,1
4,4100002.0,4100030.0,41027009.0,1
...,...,...,...,...
76211,9199422.0,4170973.0,41074080.0,1
76212,9199422.0,4176946.0,41074080.0,8
76213,9199422.0,4197464.0,41074080.0,1
76214,9199422.0,4197590.0,41074080.0,1


In [151]:
df_from_to1 = pd.merge(df_from_to1, 
             df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
             left_on='최초승차역ID', right_on='이비카드정류장ID', how='left')

df_from_to1 = pd.merge(df_from_to1, 
             df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
             left_on='최종하차역ID', right_on='이비카드정류장ID', how='left',
             suffixes=('_승차', '_하차'))

df_from_to1 = pd.merge(df_from_to1,
                       routemapping[['표준노선ID','노선명']],
                       left_on = '버스노선ID1', right_on = '표준노선ID', how = 'left',
                      )

In [152]:
cols = [col for col in df_from_to1.columns if "WGS" in col]
missing_num = df_from_to1[cols].isna().any(axis=1).sum()

print("{} / {} ({:.2f}%)".format(missing_num, len(df_from_to1), missing_num/len(df_from_to1)))

13505 / 88495 (0.15%)


In [153]:
df_from_to1 = df_from_to1[~df_from_to1[cols].isna().any(axis=1)]
df_from_to1

,최초승차역ID,최종하차역ID,버스노선ID1,이용량,이비카드정류장ID_승차,정류소명_승차,표준정류장ID_승차,WGS84위도_승차,WGS84경도_승차,관할관청_승차,이비카드정류장ID_하차,정류소명_하차,표준정류장ID_하차,WGS84위도_하차,WGS84경도_하차,관할관청_하차,표준노선ID,노선명
0,4100002.0,4100004.0,41027009.0,1,4100002.0,하북차고지,214001151.0,37.103517,127.064317,경기도 평택시,4100004.0,서탄입구,214000514.0,37.104117,127.063833,경기도 평택시,41027009.0,301
1,4100002.0,4100020.0,41027009.0,1,4100002.0,하북차고지,214001151.0,37.103517,127.064317,경기도 평택시,4100020.0,대림아파트,223000127.0,37.139800,127.068867,경기도 오산시,41027009.0,301
2,4100002.0,4100022.0,41020001.0,1,4100002.0,하북차고지,214001151.0,37.103517,127.064317,경기도 평택시,4100022.0,오산역.오산터미널,223000126.0,37.144433,127.068617,경기도 오산시,41020001.0,300
3,4100002.0,4100024.0,41027009.0,1,4100002.0,하북차고지,214001151.0,37.103517,127.064317,경기도 평택시,4100024.0,중원사거리,223000125.0,37.149850,127.068583,경기도 오산시,41027009.0,301
4,4100002.0,4100030.0,41027009.0,1,4100002.0,하북차고지,214001151.0,37.103517,127.064317,경기도 평택시,4100030.0,대우3차아파트.궐동,223000122.0,37.157767,127.067183,경기도 오산시,41027009.0,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88480,9179350.0,4108147.0,41036113.0,1,9179350.0,금곡동강남아파트,201000322.0,37.279350,126.954433,경기도 수원시,4108147.0,SK스카이뷰.SK아트리움,200000127.0,37.309350,126.988267,경기도 수원시,41036113.0,99
88481,9179350.0,4114203.0,41036115.0,1,9179350.0,금곡동강남아파트,201000322.0,37.279350,126.954433,경기도 수원시,4114203.0,수원시보훈회관.호매실마을13단지,201000410.0,37.265117,126.957300,경기도 수원시,41036115.0,15-1
88482,9179350.0,4160289.0,41036113.0,1,9179350.0,금곡동강남아파트,201000322.0,37.279350,126.954433,경기도 수원시,4160289.0,이목동.장안힐스테이트,200000151.0,37.314117,126.985000,경기도 수원시,41036113.0,99
88483,9179350.0,4179348.0,41036113.0,1,9179350.0,금곡동강남아파트,201000322.0,37.279350,126.954433,경기도 수원시,4179348.0,호매실,201000358.0,37.265950,126.952683,경기도 수원시,41036113.0,99


In [154]:
# 경기도 화성시, 경기도 오산시, 경기도 평택시, 경기도 안산시, 경기도 수원시
target_si =['경기도 화성시','경기도 오산시','경기도 평택시','경기도 안산시','경기도 수원시']

In [155]:
df_from_to1 = df_from_to1[df_from_to1['관할관청_승차'].isin(target_si) & df_from_to1['관할관청_하차'].isin(target_si)]

print(df_from_to1['관할관청_승차'].unique())
print(df_from_to1['관할관청_하차'].unique())
len(df_from_to1)

['경기도 평택시' '경기도 오산시' '경기도 화성시' '경기도 수원시' '경기도 안산시']
['경기도 평택시' '경기도 오산시' '경기도 수원시' '경기도 화성시' '경기도 안산시']


59113

In [156]:
before = len(df_from_to1)
df_from_to1.dropna(subset=['표준노선ID', '노선명'], inplace = True)
after = len(df_from_to1)
print("{} -> {} ({:.2f})".format(before, after, (before-after)/before))

59113 -> 58682 (0.01)


In [157]:
# 저장
df_from_to1.to_csv('./data/ODdata3_1.csv')